In [ ]:
!pip install datasets

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, Dataset
import collections
from transformers import RobertaTokenizer, RobertaForTokenClassification
from torch.utils.data.sampler import SubsetRandomSampler
from datasets import load_dataset
from data_loader import EventSentenceLoader
from tqdm import tqdm
import re
import numpy as np
import argparse

In [ ]:
use_freeze_model = False
validation_run = True
learning_rate = 0.00001

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


def tokenize(batched_text):
    return tokenizer(batched_text['sentence'], padding=True, truncation=True, max_length=256)


In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, data):
            self.data_points = data

    def __len__(self):
        return len(self.data_points)

    def __getitem__(self, idx):
        data = self.data_points[idx]
        return {'tokens': data['tokens'].flatten(), 'attention': data['attention'].flatten(), 'labels': torch.tensor(data['labels'])}

In [ ]:
def load_roberta_data():
    filepath = "events.txt"
    tokenizer_name = "bert-base-cased"
    loader = EventSentenceLoader(filepath, tokenizer_name)
    train_data = TextClassificationDataset(loader.load_data())

    train_size = int(0.9 * len(train_data))
    test_size = len(train_data) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(train_data, [train_size, test_size])
    train_data = list(map(lambda d: (torch.tensor(d['tokens']), d['labels'], d['attention']), train_dataset))
    test_data = list(map(lambda d: (torch.tensor(d['tokens']), d['labels'], d['attention']), test_dataset))

    return train_data, test_data

In [ ]:
def test(model, data):
    confusion_matrix_size = 2
    confusion_matrix = []

    total = 0
    correct = 0
    total_real_words = 0

    for i in range(confusion_matrix_size):
        row = []
        for j in range(confusion_matrix_size):
            row.append(0)
        confusion_matrix.append(row)

    with torch.no_grad():
        for i, batch in tqdm(enumerate(data, 0), total=len(data), leave=False):
            inputs, labels, attention = batch
            inputs, labels, attention = inputs.to(device), labels.to(device), attention.to(device)

            outputs = model(inputs).logits.to(device)

            predicted = torch.argmax(outputs, 2).flatten()
            labels = labels.flatten()
            attention = attention.flatten()

            for j in range(len(predicted)):
                if attention[j]:
                    confusion_matrix[labels[j].item()][predicted[j].item()] += 1

            total += len(predicted)
            correct += predicted.eq(labels).sum().item()
    print(f"Total Real Words: {total_real_words}")
    print("Test Accuracy: {:.3f}".format(correct/total))
    print(confusion_matrix)
    return confusion_matrix

In [ ]:
def train(model, data, epochs):
    n = len(data)

    # Define metrics to monitor change in performance during execution
    accuracy_history_epoch = []
    accuracy_history_step = []

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.9])).to(device)

    for epoch in range(1, epochs + 1):
        correct = 0
        total = 0
        for i, batch in tqdm(enumerate(data, 0), total=len(data), leave=False):
            inputs, labels, attention = batch
            inputs, labels, attention = inputs.to(device), labels.to(device), attention.to(device)

            outputs = model(inputs).logits.to(device)
            predicted = torch.argmax(outputs, 2)

            optimizer.zero_grad()

            loss = loss_func(outputs.flatten(start_dim=0, end_dim=1), F.one_hot(labels).float().flatten(start_dim=0, end_dim=1))
            loss.backward()
            optimizer.step()

            total += len(predicted)
            correct += predicted.eq(labels).sum().item()
            accuracy_history_step.append((i+1, correct/total))

        accuracy_history_epoch.append(correct / total)
        print("Epoch: {:>3d} Accuracy: {:.3f}".format(epoch, accuracy_history_epoch[-1]))

    return accuracy_history_epoch, accuracy_history_step

In [ ]:
filepath = "events.txt"
tokenizer_name = "bert-base-cased"
loader = EventSentenceLoader(filepath, tokenizer_name)
train_data = TextClassificationDataset(loader.load_data())

train_data, test_data = load_roberta_data()

<ipython-input-5-bc25106ffb53>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'tokens': data['tokens'].flatten(), 'attention': data['attention'].flatten(), 'labels': torch.tensor(data['labels'])}
<ipython-input-6-9615815f0f07>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = list(map(lambda d: (torch.tensor(d['tokens']), d['labels'], d['attention']), test_dataset))


In [ ]:
model = RobertaForTokenClassification.from_pretrained("roberta-base").to(device)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
file_path = "model_state.pth"
model.load_state_dict(torch.load(file_path))

<All keys matched successfully>

In [ ]:
indices = list(range(len(train_data)))

train_dataloader = DataLoader(train_data, batch_size=32, num_workers=0, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, num_workers=0, shuffle=0)

In [ ]:
train(model, train_dataloader, 50)

OutOfMemoryError: ignored

In [ ]:
test(model, test_dataloader)

Total Real Words: 0
Test Accuracy: 0.986
[[1626, 102], [63, 2979]]


[[1626, 102], [63, 2979]]

In [ ]:
torch.save(model.state_dict(), file_path)